# Caesar Cipher

## Introduction

In this notebook we will try to break some encryption schemes with the optimisation technique: mixed integer programming (MIP).

Julius Caesar was one of the first to use encryption when sending messages.
It is know that he used a single fixed shift for each letter in the alphabet. So say the shift is chose to be 6. Then
the letter 'a' becomes 'g', 'b' becomes 'h', 'c' becomes 'i', etc, cyclically.
This is not a very strong encryption method since only 26 'keys' are possible, namely, the shifts by the numbers 0 to 25. So one only has to try 26 shifts to decrypt the message.

So we will directly focus on trying to break a generalisation of the alphabet shift, more specifically, the alphabet permutation. Instead of 26 possible encodings, we get 26! encodings. From https://www.wolframalpha.com/input/?i=26%21 we learn that this number equals 403291461126605635584000000 or 4 times 10 to the power 26. That's a lot more possibilities to try and figure out decoding.

How do we know if we have got the right plain the message? We do this by statistical analysis of the letters occuring in the decoded message. The letter 'e' in English is the most frequent letter. So in a long enough message, the letter that 'e' is mapped onto after encoding should be the most freuqent as well. Additionally, one can also consider the frequency of each couple of subsequent letters. For example 'th' is pretty frequent in English language.

## importing libraries and setting up some ranges


In [1]:

import numpy as np
from numpy import random
import json

import string
alphabet_str = string.ascii_lowercase
print(alphabet_str)
N = len(alphabet_str)
az_list = list(alphabet_str)
print(az_list)

abcdefghijklmnopqrstuvwxyz
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## A fixed permutation

### Setting up a random fixed permutation

In [2]:
# As for the encoding schema, 
# let's start with a general fixed letter to fixed letter permutation first.

# Choose one of the 26! letter permutations randomly. 
# With a fixed seed parameter the same one can be chosen repeatedly.
def construct_random_letter_permutation(seed=None):
    if seed != None:
        random.seed(seed)
    letter_indices = np.array(list(range(0,len(alphabet_str))))
    #print(letter_indices)
    letter_indices_permutated = letter_indices.copy()
    letter_indices_permutated = random.permutation(letter_indices_permutated)
    #print(letter_indices_permutated)    
    letter_permutation_dict = {}
    for letter_idx_in in letter_indices:
        letter_idx_out = letter_indices_permutated[letter_idx_in]
        letter_permutation_dict[alphabet_str[letter_idx_in]] = alphabet_str[letter_idx_out] 
    return letter_permutation_dict

### Encoding: Applying a fixed permutation

In [3]:
# enoode a message with a letter permutation
def encode(msg, letter_permutation):
    enc_msg = ''
    for i in range(len(msg)):
        enc_msg += letter_permutation[msg[i]]
    return enc_msg

In [4]:
plaintext_msg = 'Let us try to decrypt with a quantum computer.'
# https://www.englishclub.com/reading/cr-hare-tortoise.htm
plaintext_msg = """
The Hare and the Tortoise.
One day the Hare laughed at the short feet and slow speed of the Tortoise. The Tortoise replied:
You may be as fast as the wind, but I will beat you in a race!
The Hare thought this idea was impossible and he agreed to the proposal. 
It was agreed that the Fox should choose the course and decide the end.
The day for the race came, and the Tortoise and Hare started together.
The Tortoise never stopped for a moment, walking slowly but steadily, 
right to the end of the course. The Hare ran fast and stopped to lie down for a rest. 
But he fell fast asleep. Eventually, he woke up and ran as fast as he could. 
But when he reached the end, he saw the Tortoise there already, sleeping comfortably after her effort.'
"""

# Remove spaces and dots, and restrict to lowercase characters only, so we have 26 symbols.
plaintext_msg = plaintext_msg.replace(' ', '').replace('.', '').replace('\n', '').replace(',', '')
plaintext_msg = plaintext_msg.replace(':', '').replace('!', '').replace('\'', '').lower()

print('plaintext message = {:s}'.format(plaintext_msg))
letter_permutation_dict = construct_random_letter_permutation(3)
print('letter permutation dictionary =')
print(json.dumps(letter_permutation_dict, indent=2))
enc_msg = encode(plaintext_msg, letter_permutation_dict)
print('encoded message = {:s}'.format(enc_msg))

plaintext message = thehareandthetortoiseonedaytheharelaughedattheshortfeetandslowspeedofthetortoisethetortoiserepliedyoumaybeasfastasthewindbutiwillbeatyouinaracetheharethoughtthisideawasimpossibleandheagreedtotheproposalitwasagreedthatthefoxshouldchoosethecourseanddecidetheendthedayfortheracecameandthetortoiseandharestartedtogetherthetortoiseneverstoppedforamomentwalkingslowlybutsteadilyrighttotheendofthecoursethehareranfastandstoppedtoliedownforarestbuthefellfastasleepeventuallyhewokeupandranasfastashecouldbutwhenhereachedtheendhesawthetortoisetherealreadysleepingcomfortablyafterhereffort
letter permutation dictionary =
{
  "a": "s",
  "b": "r",
  "c": "m",
  "d": "x",
  "e": "p",
  "f": "q",
  "g": "n",
  "h": "c",
  "i": "b",
  "j": "w",
  "k": "o",
  "l": "e",
  "m": "v",
  "n": "g",
  "o": "h",
  "p": "f",
  "q": "u",
  "r": "j",
  "s": "l",
  "t": "z",
  "u": "t",
  "v": "a",
  "w": "i",
  "x": "d",
  "y": "y",
  "z": "k"
}
encoded message = zcpcsjpsgxzcpzhjzhblphgpxsyzcpcsjpes

### Decoding: inverting a permutation

Assuming we know the encoding, so the forward permutation, we can check that the backwards permutation, which consists of swapping rows for columns generates the original plaintext message.

In [5]:

inverse_letter_permutation_dict = {v: k for k, v in letter_permutation_dict.items()}
print('inverse letter permutation dictionary =')
print(json.dumps(inverse_letter_permutation_dict, indent=2))
recovered_msg = encode(enc_msg, inverse_letter_permutation_dict)
print('recovered message = {:s}'.format(recovered_msg))

inverse letter permutation dictionary =
{
  "s": "a",
  "r": "b",
  "m": "c",
  "x": "d",
  "p": "e",
  "q": "f",
  "n": "g",
  "c": "h",
  "b": "i",
  "w": "j",
  "o": "k",
  "e": "l",
  "v": "m",
  "g": "n",
  "h": "o",
  "f": "p",
  "u": "q",
  "j": "r",
  "l": "s",
  "z": "t",
  "t": "u",
  "a": "v",
  "i": "w",
  "d": "x",
  "y": "y",
  "k": "z"
}
recovered message = thehareandthetortoiseonedaytheharelaughedattheshortfeetandslowspeedofthetortoisethetortoiserepliedyoumaybeasfastasthewindbutiwillbeatyouinaracetheharethoughtthisideawasimpossibleandheagreedtotheproposalitwasagreedthatthefoxshouldchoosethecourseanddecidetheendthedayfortheracecameandthetortoiseandharestartedtogetherthetortoiseneverstoppedforamomentwalkingslowlybutsteadilyrighttotheendofthecoursethehareranfastandstoppedtoliedownforarestbuthefellfastasleepeventuallyhewokeupandranasfastashecouldbutwhenhereachedtheendhesawthetortoisetherealreadysleepingcomfortablyafterhereffort


## Guessing the encoding permutation: A variable Permutation

The permutation is essentially captured by a 26 time 26 boolean matrix.
It has a 1 in row 'e', column 'h' if the encoding maps 'e' to 'h'.


### Core variables

In [6]:
import gurobipy as gp
from gurobipy import GRB
m = gp.Model()

print('ord(a) = {:d}'.format(ord('a')))
#p = m.addMVar((N,N), vtype=GRB.BINARY) # p[a][b] is the encoding permutation -> p[b][a] encodes
p = m.addVars(az_list, az_list, vtype=GRB.BINARY, name='p') # p[a][b] is the encoding permutation -> p[b][a] encodes

Using license file /Library/gurobi910/gurobi.lic
ord(a) = 97


### Constraints

In [7]:
for y in az_list:
    row_sum = 0
    for x in az_list:
        row_sum += p[y, x]
    m.addConstr(row_sum == 1, 'row_{:s}_sum_is_1'.format(y))
for x in az_list:
    col_sum = 0
    for y in az_list:
        col_sum += p[y, x]
    m.addConstr(col_sum == 1, 'col_{:s}_sum_is_1'.format(x))

# relation between original letter and encrypted letter
for msg_let_idx, enc_let in enumerate(enc_msg):
    enc_let_ord = ord(enc_let) - ord('a')
    assert enc_let_ord <= N-1
    assert enc_let_ord >= 0
    print('msg_let_idx:{:d}, enc_let:{:s}, enc_let_ord:{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
    expr = 0
    for x in az_list:
        bit = p[enc_let, x]
        expr += bit * (ord(x) - ord('a'))
    m.addConstr(expr <= N-1, 'le_msg_let_idx_{:d}_enc_let_{:s}_enc_let_ord_{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
    m.addConstr(expr >= 0, 'ge_msg_let_idx_{:d}_enc_let_{:s}_enc_let_ord_{:d}'.format(msg_let_idx, enc_let, enc_let_ord))

#p_dec = m.addVars(permutation_dec, lb=[0]*N, ub=[25]*N, vtype=GRB.BOOL, name='p')  
m.write('decode.lp')
m.optimize()

msg_let_idx:0, enc_let:z, enc_let_ord:25
msg_let_idx:1, enc_let:c, enc_let_ord:2
msg_let_idx:2, enc_let:p, enc_let_ord:15
msg_let_idx:3, enc_let:c, enc_let_ord:2
msg_let_idx:4, enc_let:s, enc_let_ord:18
msg_let_idx:5, enc_let:j, enc_let_ord:9
msg_let_idx:6, enc_let:p, enc_let_ord:15
msg_let_idx:7, enc_let:s, enc_let_ord:18
msg_let_idx:8, enc_let:g, enc_let_ord:6
msg_let_idx:9, enc_let:x, enc_let_ord:23
msg_let_idx:10, enc_let:z, enc_let_ord:25
msg_let_idx:11, enc_let:c, enc_let_ord:2
msg_let_idx:12, enc_let:p, enc_let_ord:15
msg_let_idx:13, enc_let:z, enc_let_ord:25
msg_let_idx:14, enc_let:h, enc_let_ord:7
msg_let_idx:15, enc_let:j, enc_let_ord:9
msg_let_idx:16, enc_let:z, enc_let_ord:25
msg_let_idx:17, enc_let:h, enc_let_ord:7
msg_let_idx:18, enc_let:b, enc_let_ord:1
msg_let_idx:19, enc_let:l, enc_let_ord:11
msg_let_idx:20, enc_let:p, enc_let_ord:15
msg_let_idx:21, enc_let:h, enc_let_ord:7
msg_let_idx:22, enc_let:g, enc_let_ord:6
msg_let_idx:23, enc_let:p, enc_let_ord:15
msg_let_idx:2

Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1208 rows, 676 columns and 30252 nonzeros
Model fingerprint: 0x7fff7bc2
Variable types: 0 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [8]:
def print_permutation(permutation):
    print('  ' + ' '.join([l for l in az_list]))
    for r_l in az_list:
        print(r_l, end=' ')
        for c_l in az_list:
            bit = int(permutation[r_l, c_l].x)
            s = '1' if bit==1 else ' '
            print(s, end=' ')
        print()
print_permutation(p)

  a b c d e f g h i j k l m n o p q r s t u v w x y z
a                                             1       
b                     1                               
c                                                 1   
d                                 1                   
e                                           1         
f         1                                           
g                                       1             
h               1                                     
i       1                                             
j                                               1     
k 1                                                   
l                                   1                 
m   1                                                 
n                         1                           
o                             1                       
p                 1                                   
q                   1                                 
r     1    

In [9]:
# two letters that are the same in the encrypted message originate from two letters in the 
# original plain text message that are also the same.
# This is already embedded (hard coded) in the way the permutation constraitn was formulated.

# unigram statistics

# bigram statistics


In [10]:
# Reconstruct the message from the permutation matrix found.
def codec(in_msg, decode_io_encode):  # encode is col to row letters
    out_msg = ''
    for msg_let_idx, in_let in enumerate(in_msg):
        in_let_ord = ord(in_let)-ord('a')
        assert in_let_ord < 26
        assert in_let_ord >= 0
        #print('msg_let_idx:{:d}, in_let:{:s}, in_let_ord:{:d}'.format(msg_let_idx, in_let, in_let_ord))
        
        out_let_ord = 0
        for out_let in az_list:
            if decode_io_encode == False:
                bit = int(p[in_let, out_let].x)  # encode 'forward' matrix use
            else:
                bit = int(p[out_let, in_let].x)  # decode ~ backward matrix use
                # The inverse matrix of a permutation matrix is its transposed.
                
            out_let_ord += bit * (ord(out_let) - ord('a'))
            
        out_let = chr(ord('a') + out_let_ord)
        out_msg += out_let
    return out_msg
    
print('enc_msg: {:s} has length {:d}.'.format(enc_msg, len(enc_msg)))

dec_msg = codec(enc_msg, decode_io_encode=True)  # backwards
print('dec_msg: {:s} has length {:d}.'.format(dec_msg, len(dec_msg)))
assert len(enc_msg) == len(dec_msg)

enc_again_msg = codec(dec_msg, decode_io_encode=False)  # forwards
print('enc_again_msg: {:s} has length {:d}.'.format(enc_again_msg, len(enc_again_msg)))
assert len(enc_again_msg) == len(dec_msg)

enc_msg: zcpcsjpsgxzcpzhjzhblphgpxsyzcpcsjpestncpxszzcplchjzqppzsgxlehilfppxhqzcpzhjzhblpzcpzhjzhblpjpfebpxyhtvsyrpslqslzslzcpibgxrtzbibeerpszyhtbgsjsmpzcpcsjpzchtnczzcblbxpsislbvfhllbrepsgxcpsnjppxzhzcpfjhfhlsebzislsnjppxzcszzcpqhdlchtexmchhlpzcpmhtjlpsgxxpmbxpzcppgxzcpxsyqhjzcpjsmpmsvpsgxzcpzhjzhblpsgxcsjplzsjzpxzhnpzcpjzcpzhjzhblpgpapjlzhffpxqhjsvhvpgziseobgnlehieyrtzlzpsxbeyjbnczzhzcppgxhqzcpmhtjlpzcpcsjpjsgqslzsgxlzhffpxzhebpxhigqhjsjplzrtzcpqpeeqslzsleppfpapgztseeycpihoptfsgxjsgslqslzslcpmhtexrtzicpgcpjpsmcpxzcppgxcplsizcpzhjzhblpzcpjpsejpsxyleppfbgnmhvqhjzsreysqzpjcpjpqqhjz has length 578.
dec_msg: wrsrtqstvjwrswhqwhmyshvsjtcwrsrtqsftgzrsjtwwrsyrhqwdsswtvjyfhpyxssjhdwrswhqwhmyswrswhqwhmysqsxfmsjchgetclstydtywtywrspmvjlgwmpmfflstwchgmvtqtnswrsrtqswrhgzrwwrmymjstptymexhyymlfstvjrstzqssjwhwrsxqhxhytfmwptytzqssjwrtwwrsdhiyrhgfjnrhhyswrsnhgqystvjjsnmjswrssvjwrsjtcdhqwrsqtnsntestvjwrswhqwhmystvjrtqsywtqwsjwhzswrsqwrswhqwhmysvsksqywhxxsjdhqtehesvwptfomvzyfhpfclgwywstjmfcqmzrwwhwrssvjhd

### Adding unigram statistics

In [11]:
# now we add a goal function that takes care of maximum likelihood of frequencey of letters.
# unigram: deviate as little as possible from the normal statistical letter distribution.
# u['a'] counts the number of 'a's in the decoded message, given:
#   the fixed encoded message enc_msg
#   and given the variable permutation p (transforming it into dec_msg)
unigram_count = m.addVars(az_list, vtype=GRB.CONTINUOUS, name='uc')
for dec_let in az_list:  # x is the decoded letter
    count_expr = 0
    for msg_let_idx, enc_let in enumerate(enc_msg):
        enc_let_ord = ord(enc_let)-ord('a')
        assert enc_let_ord < 26
        assert enc_let_ord >= 0
        #print('msg_let_idx:{:d}, enc_let:{:s}, enc_let_ord:{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
        bit = p[dec_let, enc_let]  # encoding is forward use of matrix
        count_expr += bit
    m.addConstr(unigram_count[dec_let] == count_expr, 'unigram_count_{:s}'.format(dec_let))
    
    
# From view-source:http://norvig.com/mayzner.html, we get the unigrams data
english_std_unigram = { 'e': 12.49, 't': 9.28, 'a': 8.04, 'o':7.64, 'i': 7.57, 'n':7.23, 's': 6.51, 'r': 6.28, 'h': 5.05, 'l':4.07, 'd':3.82, 'c':3.34, 'u':2.73, 'm': 2.51, 'f':2.40, 'p': 2.14, 'g': 1.87, 'w': 1.68, 'y': 1.66, 'b': 1.48, 'v':1.05, 'k': 0.54, 'x': 0.23, 'j': 0.16, 'q': 0.12, 'z': 0.09 }
total = 0.0
for k, v in english_std_unigram.items():
    total += v
total, len(english_std_unigram.keys())

# say: permutation as ints. p[f] = t or 1. f/t = from/to, f in [0,25], t in [0, 25].
# constraints: avoiding the ones we'd have to impose for bool vars:
# say p is the decode permutation, mapping encoded letter to decoded one. p is to be found var.
# sum_f p[f][t] = 1 for all t
# sum_t p[f][t] = 1 for all f
# objective: achieve typical fraction for all letters.
# Minimize quadratic errors = energy function?
#sum_{l in enc_text} sum_o (f[p[ord(l)-ord('a')][]] - f[])^2
    
unigram_obj = 0
for dec_let in az_list:
    should_occur_per_100_letters = english_std_unigram[dec_let]
    does_occur_per_100_letters = unigram_count[dec_let] * 100 / len(enc_msg)
    diff = does_occur_per_100_letters - should_occur_per_100_letters
    diff_sq = diff * diff
    unigram_obj += diff_sq

m.setObjective(unigram_obj, GRB.MINIMIZE)

m.write('decode_unigram.lp')

m.setParam('MIPGap', 0.001)
m.setParam('Timelimit', 300)

m.optimize()
#print_permutation(p)

Changed value of parameter MIPGap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter Timelimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1234 rows, 702 columns and 30876 nonzeros
Model fingerprint: 0x5bd86502
Model has 26 quadratic objective terms
Variable types: 26 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [3e-02, 4e+00]
  QObjective range [6e-02, 6e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

MIP start from previous solve produced solution with objective 667.916 (0.01s)
Loaded MIP start from previous solve with objective 667.916

Presolve removed 1156 rows and 0 columns
Presolve time: 0.01s
Presolved: 78 rows, 702 columns, 1976 nonzeros
Presolved model has 26 quadratic objective te

In [12]:
def print_unigram_count(unigram):
    print('    ' + '  '.join([l for l in az_list]))
    print('   ', end='')
    total_count = 0
    for l in az_list:
        count = int(unigram[l].x)
        total_count += count
        print('{:02d}'.format(count), end=' ')
    print()
    print('total letter count = {:d}.'.format(total_count))
    
print_unigram_count(unigram_count)

    a  b  c  d  e  f  g  h  i  j  k  l  m  n  o  p  q  r  s  t  u  v  w  x  y  z
   53 06 16 22 93 11 10 23 46 00 02 22 12 37 46 10 00 31 36 69 14 02 08 01 08 00 
total letter count = 578.


In [13]:
print_permutation(p)

  a b c d e f g h i j k l m n o p q r s t u v w x y z
a                                     1               
b                                           1         
c                                 1                   
d         1                                           
e                               1                     
f                 1                                   
g                         1                           
h             1                                       
i               1                                     
j                     1                               
k                             1                       
l   1                                                 
m           1                                         
n                   1                                 
o     1                                               
p                                                 1   
q                                         1           
r          

In [14]:
print('plain text msg = {:s}'.format(plaintext_msg))

dec_msg = codec(enc_msg, decode_io_encode=True)  # backwards
print('dec_msg: {:s} has length {:d}.'.format(dec_msg, len(dec_msg)))
assert len(enc_msg) == len(dec_msg)


plain text msg = thehareandthetortoiseonedaytheharelaughedattheshortfeetandslowspeedofthetortoisethetortoiserepliedyoumaybeasfastasthewindbutiwillbeatyouinaracetheharethoughtthisideawasimpossibleandheagreedtotheproposalitwasagreedthatthefoxshouldchoosethecourseanddecidetheendthedayfortheracecameandthetortoiseandharestartedtogetherthetortoiseneverstoppedforamomentwalkingslowlybutsteadilyrighttotheendofthecoursethehareranfastandstoppedtoliedownforarestbuthefellfastasleepeventuallyhewokeupandranasfastashecouldbutwhenhereachedtheendhesawthetortoisetherealreadysleepingcomfortablyafterhereffort
dec_msg: toeoaneahrtoetintilseiheraptoeoanedauyoerattoesointceetahrsdifsmeerictoetintilsetoetintilsenemdlerpiubapweascastastoeflhrwutlflddweatpiulhanagetoeoanetoiuyottolslreafaslbmisslwdeahroeayneertitoemnimisadltfasayneertoattoecixsoiudrgoiisetoegiunseahrreglretoeehrtoerapcintoenagegabeahrtoetintilseahroanestantertiyetoentoetintilsehevenstimmercinabibehtfadklhysdifdpwutstearldpnlyottitoeehrictoegiuns

Clearly this is not the original message yet. But also we get closer to it since already some letters are correct. Like we get 'toe i.o. 'the', that's 2 out of 3 letters correct. But we get 'oane' i.o. 'hare', that's only 1 in 4 letters correct. So we will need some more constraints. Let's add a stimulus in the direction of bigrams statistics.

### Adding bigram statistics

We extract the frequency of each two consecutive letters (in words) from research perfomed by Mayzner and available at
file:///Users/peter/projects/OnComputation.github/QuantumVsEnigma/mayzner.html

The section on bigram frequncies has data encoded like: 

<tr><td title="AA: 0.003%; 79,794,787"><img src="o.jpg" height=16 width=1><span style="position:relative; left:1; bottom:4">AA</span></span><td title="BA: 0.146%; 4,122,472,992"><img ...

So we fetch this with a script using regular epressions.



In [15]:
import re

file1 = open('bigrams.html', 'r')
lines = file1.readlines()
 
count = 0
# Strips the newline character
total = 0
std_bigram = {}
total_bigrams = 2.8e12 # 2.8 trillion mentions
for line in lines:
    count += 1
    #print("Line{}: {}".format(count, line.strip()))
    match = re.findall(r"title=\"(\S\S): (\d+.\d+)%; (\d+,\d+,\d+|\d+,\d+|\d+)",line)
    #print(r1)    
    total += len(match)
    #print(len(m))
    #sum_prob = 0.0
    for triplet in match:
        #print(triplet)
        bigram_letter_0 = triplet[0].lower()[0]
        bigram_letter_1 = triplet[0].lower()[1]
        bigram_seq_prob = float(triplet[1])  # compared to all bigrams with same first letter (?)
        bigram_couple_occur = float(triplet[2].replace(',',''))  # compared to all 26*26 couples (surely)
        if bigram_letter_0 not in std_bigram.keys():
            std_bigram[bigram_letter_0] = {}
        if bigram_letter_1 not in std_bigram[bigram_letter_0].keys():
            #std_bigram[bigram_letter_0][bigram_letter_1] = bigram_seq_prob
            std_bigram[bigram_letter_0][bigram_letter_1] = bigram_couple_occur #/ total_bigrams
            #sum_prob += bigram_seq_prob
            #if bigram_letter_1 == 'z':
            #    print('  -> sum_prob = {:f}'.format(sum_prob))
        else:
            assert False # no bigram should accur more than once
print('total found ({:d}) is {:d} short of {:d}.'.format(total, 26*26 - total, 26*26))
#print(std_bigram)

total found (676) is 0 short of 676.


In [16]:
# check:
sum_occur = 0.0
for l0 in std_bigram.keys():
    for l1 in std_bigram[l0].keys():
        occur = std_bigram[l0][l1]
        sum_occur += occur
for l0 in std_bigram.keys():
    for l1 in std_bigram[l0].keys():
        std_bigram[l0][l1] /= sum_occur

total_occur = sum_occur
sum_occur = 0
for l0 in std_bigram.keys():
    for l1 in std_bigram[l0].keys():
        occur = std_bigram[l0][l1]
        sum_occur += occur
print('sum_prob = {:f}.'.format(sum_occur))

sum_prob = 1.000000.


In [17]:
bigram_count = m.addVars(az_list, az_list, vtype=GRB.CONTINUOUS, name='bc')
bigram_bit = m.addVars(az_list, az_list, list(range(0, len(enc_msg)-1)), vtype=GRB.CONTINUOUS, name='bb')

for dec_let0 in az_list:  # x is the first decoded letter
    for dec_let1 in az_list:  # x is the second decoded letter
        count_expr = 0
        msg_idx = 0
        for enc_let0, enc_let1 in zip(list(enc_msg[1:]), list(enc_msg[:-1])):
            enc_let0_ord = ord(enc_let0)-ord('a')
            assert enc_let_ord < 26
            assert enc_let_ord >= 0
            enc_let1_ord = ord(enc_let1)-ord('a')
            assert enc_let1_ord < 26
            assert enc_let1_ord >= 0
            #print('msg_let_idx:{:d}, enc_let:{:s}, enc_let_ord:{:d}'.format(msg_let_idx, enc_let, enc_let_ord))
            bit_let0 = p[dec_let0, enc_let0]  # encoding is forward use of matrix
            bit_let1 = p[dec_let1, enc_let1]  # encoding is forward use of matrix
            m.addGenConstrAnd(bigram_bit[dec_let0, dec_let1, msg_idx], [bit_let0, bit_let1], 
                              'bb_{:s}_{:s}_{:d}_and'.format(dec_let0, dec_let1, msg_idx))
            count_expr += bigram_bit[dec_let0, dec_let1, msg_idx]  # TODO: still to correct! to AND io +
            msg_idx += 1
            
        and_constr_name = 'bigram_count_{:s}_{:s}'.format(dec_let0, dec_let1)
        #print(and_constr_name)
        m.addConstr(bigram_count[dec_let0, dec_let1] == count_expr, and_constr_name)

threshold_prob = 0.00001
bigram_obj = 0
n_bigram_obj_terms = 0
for dec_let0 in az_list:  # x is the first decoded letter
    for dec_let1 in az_list:  # x is the second decoded letter
        if std_bigram[dec_let0][dec_let1] >= threshold_prob:
            should_occur_per_letter_couple = std_bigram[dec_let0][dec_let1]
            does_occur_per_letter_couple = bigram_count[dec_let0, dec_let1] / len(enc_msg)
            diff = does_occur_per_letter_couple - should_occur_per_letter_couple
            diff_sq = diff * diff
            bigram_obj += diff_sq
            n_bigram_obj_terms += 1

print('n_bigram_obj_terms = {:d}'.format(n_bigram_obj_terms))

m.setObjective(unigram_obj + bigram_obj, GRB.MINIMIZE)

m.write('decode_unigram_bigram.lp')

m.setParam('MIPGap', 0.00001)
m.setParam('Timelimit', 300)

m.optimize()
#print_permutation(p)

n_bigram_obj_terms = 636
Changed value of parameter MIPGap to 1e-05
   Prev: 0.001  Min: 0.0  Max: inf  Default: 0.0001
Parameter Timelimit unchanged
   Value: 300.0  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1910 rows, 391430 columns and 421604 nonzeros
Model fingerprint: 0xf892c758
Model has 662 quadratic objective terms
Model has 390052 general constraints
Variable types: 390754 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [4e-08, 4e+00]
  QObjective range [6e-06, 6e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

MIP start from previous solve produced solution with objective 26.3826 (0.45s)
Loaded MIP start from previous solve with objective 26.3826

Presolve removed 6789 rows and 23692 columns (presolve time = 5s) ...
Presolve removed 6789 rows 



## Conclusion

...
